In [1]:
import pandas as pd
import geopandas as gpd
import os

In [50]:
445_000.0/999

445.44544544544544

In [49]:
gdf[gdf['count'] == gdf['count'].max()]

,community,area,shape_area,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len,Final_Brokerage,count,Price,brokerage_info,geometry
5005,NEAR NORTH SIDE,0,76675895.9728,0,8,8,0,0,57293.1649516,@properties Christie's International Real Estate,999,445000.0,"2 Independent Realty LLC (1), 312 Estates LLC ...","MULTIPOLYGON (((-87.62446 41.91157, -87.62459 ..."


In [2]:
gdf = gpd.read_file("brokerage_community_data.geojson")

In [5]:
gdf_boundaries = gpd.read_file("Boundaries - Community Areas (current).geojson")

Looking for a heat map by total sales volume, not by number of deals (not sure if that is how you did it already or if this would require a change)<br><br>
Please get rid of the "median price" displayed for each community area and can we instead display "total sales volume" for each community area. We would still like to see each area's total number of deals as well<br><br>
Where we show the top brokerage in each area and then their sales, it reads a bit confusing. Instead of labeling it "Top Brokerage" and "Number of Brokerage Sales," can we instead label them "Top Brokerage" and "Top Brokerage's Sales" - or "Top Brokerage Number of Sales"

In [6]:
count_sum_gdf = gdf.groupby("community").agg({'count':'sum','Price':'sum'})

In [19]:
count_sum_gdf_merged = pd.merge(left=count_sum_gdf, left_on="community", how="inner", right=gdf_boundaries,
                               right_on="community")

In [21]:
gdf_boundaries.columns

Index(['community', 'area', 'shape_area', 'perimeter', 'area_num_1',
       'area_numbe', 'comarea_id', 'comarea', 'shape_len', 'geometry'],
      dtype='object')

In [23]:
count_sum_gdf_merged_final = count_sum_gdf_merged.drop(columns=['area', 'shape_area', 'perimeter', 'area_num_1',
       'area_numbe', 'comarea_id', 'comarea', 'shape_len'])

In [46]:
gdf_sorted

,community,area,shape_area,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len,Final_Brokerage,count,Price,brokerage_info,geometry
1213,ALBANY PARK,0,53542230.8191,0,14,14,0,0,39339.0164387,@properties Christie's International Real Estate,100,485000.0,@properties Christie's International Real Esta...,"MULTIPOLYGON (((-87.70404 41.97355, -87.70403 ..."
1247,ALBANY PARK,0,53542230.8191,0,14,14,0,0,39339.0164387,Compass,60,477500.0,@properties Christie's International Real Esta...,"MULTIPOLYGON (((-87.70404 41.97355, -87.70403 ..."
1256,ALBANY PARK,0,53542230.8191,0,14,14,0,0,39339.0164387,Dream Town Real Estate,54,672500.0,@properties Christie's International Real Esta...,"MULTIPOLYGON (((-87.70404 41.97355, -87.70403 ..."
1220,ALBANY PARK,0,53542230.8191,0,14,14,0,0,39339.0164387,Baird & Warner,32,442500.0,@properties Christie's International Real Esta...,"MULTIPOLYGON (((-87.70404 41.97355, -87.70403 ..."
1246,ALBANY PARK,0,53542230.8191,0,14,14,0,0,39339.0164387,Coldwell Banker Realty,32,434000.0,@properties Christie's International Real Esta...,"MULTIPOLYGON (((-87.70404 41.97355, -87.70403 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,WOODLAWN,0,57815179.512,0,42,42,0,0,46936.9592443,Tabs Realty Group LLC,1,594000.0,"33 Realty (1), 5th Group Corp (1), @properties...","MULTIPOLYGON (((-87.57714 41.78615, -87.57714 ..."
733,WOODLAWN,0,57815179.512,0,42,42,0,0,46936.9592443,"The Margie Smigel Group, LLC",1,305000.0,"33 Realty (1), 5th Group Corp (1), @properties...","MULTIPOLYGON (((-87.57714 41.78615, -87.57714 ..."
734,WOODLAWN,0,57815179.512,0,42,42,0,0,46936.9592443,Top Care Realty LLC,1,155000.0,"33 Realty (1), 5th Group Corp (1), @properties...","MULTIPOLYGON (((-87.57714 41.78615, -87.57714 ..."
735,WOODLAWN,0,57815179.512,0,42,42,0,0,46936.9592443,"Turn-Key Realty, LLC",1,815000.0,"33 Realty (1), 5th Group Corp (1), @properties...","MULTIPOLYGON (((-87.57714 41.78615, -87.57714 ..."


In [10]:
gdf_sorted = gdf.sort_values(by=["community","count"], ascending=[True,False])

top_brokerages = gdf_sorted.groupby("community").first().reset_index()

In [14]:
top_brokerages_final = top_brokerages[["community","Final_Brokerage","count","Price","geometry"]]

In [25]:
# Check if it's already a GeoDataFrame
if not isinstance(count_sum_gdf_merged_final, gpd.GeoDataFrame):
    # If it's a DataFrame, convert it to a GeoDataFrame and specify the geometry column
    count_sum_gdf_merged_final = gpd.GeoDataFrame(count_sum_gdf_merged_final, geometry='geometry')

# Now save it to GeoJSON
count_sum_gdf_merged_final.to_file("count_sum_gdf.geojson", driver="GeoJSON")

In [26]:
top_brokerages_final.to_file("top_brokerages.geojson",driver="GeoJSON")

In [28]:
count_sum_gdf_merged_final

,community,count,Price,geometry
0,ALBANY PARK,620,63971905.0,"MULTIPOLYGON (((-87.70404 41.97355, -87.70403 ..."
1,ARCHER HEIGHTS,116,18485050.0,"MULTIPOLYGON (((-87.71437 41.82604, -87.71436 ..."
2,ARMOUR SQUARE,90,15671900.0,"MULTIPOLYGON (((-87.62917 41.84556, -87.62947 ..."
3,ASHBURN,554,48842681.0,"MULTIPOLYGON (((-87.71255 41.75734, -87.71252 ..."
4,AUBURN GRESHAM,581,49005008.0,"MULTIPOLYGON (((-87.63990 41.75615, -87.63990 ..."
...,...,...,...,...
72,WEST LAWN,260,31971632.0,"MULTIPOLYGON (((-87.71255 41.75734, -87.71278 ..."
73,WEST PULLMAN,411,32671425.0,"MULTIPOLYGON (((-87.61828 41.65911, -87.61829 ..."
74,WEST RIDGE,807,59002067.0,"MULTIPOLYGON (((-87.68465 42.01948, -87.68464 ..."
75,WEST TOWN,2808,169075378.5,"MULTIPOLYGON (((-87.65686 41.91078, -87.65685 ..."


In [29]:
top_brokerages_final = top_brokerages_final.rename(columns={'count':'brokerage_count','Price':'brokerage_price'})


In [31]:
final_merged_gdf = pd.merge(left=count_sum_gdf_merged_final, left_on='community', how='inner',
                           right=top_brokerages_final, right_on='community')

In [35]:
final_merged_gdf = final_merged_gdf.drop(columns='geometry_x')
final_merged_gdf = final_merged_gdf.rename(columns={'geometry_y':'geometry'})

In [36]:
final_merged_gdf.to_file("final_merged_gdf.geojson",driver="GeoJSON")

In [39]:
final_merged_gdf['Price'].max()

169075378.5

In [41]:
169075378.5/8

21134422.3125

In [40]:
final_merged_gdf['Price'].min()

640000.0

In [62]:
final_merged_gdf.groupby('Final_Brokerage')['brokerage_count'].sum()

Final_Brokerage
@properties Christie's International Real Estate    4790
Berkshire Hathaway HomeServices Chicago               81
Bloom/Sanchez Realty, Inc.                            42
Century 21 S.G.R., Inc.                               13
Coldwell Banker Realty                               793
Compass                                             1169
Healy Real Estate                                     35
Kale Realty                                            9
Keller Williams ONEChicago                             5
RE/MAX MI CASA                                       314
Realty of Chicago LLC                                 54
Name: brokerage_count, dtype: int64

In [51]:
485000.0/100

4850.0

In [60]:
len(gdf)

10370

In [59]:
gdf['count'].sum()

47236

In [55]:
print(f'${final_merged_gdf["Price"].sum():,}')

$3,789,985,326.0


In [43]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CookCountyNeighborhoods_FiniteMap_10_30_24
